In [1]:
pass

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1606210490394_0005,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
import os
import sys
import json
# from pyspark.sql import SparkSession

import datetime

import boto3

from s3fs import S3FileSystem
fs = S3FileSystem()

from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col

from urllib.parse import urlparse

# Parameters

In [3]:
path_cookie_cookie = 's3://zeta-dc-ml/ai-audiences/datacloud_cookie_cookie_relations/cookie_cookie_staging/dt=2020-11-16'
path_cookie_email = 's3://zeta-dc-ml/ai-audiences/datacloud_cookie_emails_links_export/cookie_email_staging/dt=2020-11-16'



In [4]:
path_UI_COO_disqus = 's3://zeta-dc-ml/ai-audiences/user_activity_daily_disqus_parquet_UI_COOmatrix/'
path_UI_COO_sizmek = 's3://zeta-dc-ml/ai-audiences/user_activity_daily_sizmek_parquet_UI_COOmatrix/'
path_UI_COO_zync = 's3://zeta-dc-ml/ai-audiences/user_activity_daily_zync_parquet_UI_COOmatrix/'


In [5]:
path_UI_interactions_baseline_disqus = 's3://shubh-sandbox/ai-audiences/q4/10_path_UI_interactions_baseline_disqus_90'
path_UI_interactions_baseline_zync = 's3://shubh-sandbox/ai-audiences/q4/10_path_UI_interactions_baseline_zync_90'


path_UI_interactions_disqus = 's3://shubh-sandbox/ai-audiences/q4/11_path_UI_interactions_disqus_90'
path_UI_interactions_sizmek = 's3://shubh-sandbox/ai-audiences/q4/11_path_UI_interactions_sizmek_90'
path_UI_interactions_zync = 's3://shubh-sandbox/ai-audiences/q4/11_path_UI_interactions_zync_90'


path_UI_interactions_email_disqus = 's3://shubh-sandbox/ai-audiences/q4/12_path_UI_interactions_email_disqus_90'
path_UI_interactions_email_sizmek = 's3://shubh-sandbox/ai-audiences/q4/12_path_UI_interactions_email_sizmek_90'
path_UI_interactions_email_zync = 's3://shubh-sandbox/ai-audiences/q4/12_path_UI_interactions_email_zync_90'


path_UI_interactions_cookie_only_disqus = 's3://shubh-sandbox/ai-audiences/q4/21_path_UI_interactions_cookie_only_disqus_90'
path_UI_interactions_cookie_only_sizmek = 's3://shubh-sandbox/ai-audiences/q4/21_path_UI_interactions_cookie_only_sizmek_90'
path_UI_interactions_cookie_only_zync = 's3://shubh-sandbox/ai-audiences/q4/21_path_UI_interactions_cookie_only_zync_90'


path_UI_interactions_email_only_disqus = 's3://shubh-sandbox/ai-audiences/q4/22_path_UI_interactions_email_only_disqus_90'
path_UI_interactions_email_only_sizmek = 's3://shubh-sandbox/ai-audiences/q4/22_path_UI_interactions_email_only_sizmek_90'
path_UI_interactions_email_only_zync = 's3://shubh-sandbox/ai-audiences/q4/22_path_UI_interactions_email_only_zync_90'


path_UI_interactions_cookie_only_cookie_map_disqus = 's3://shubh-sandbox/ai-audiences/q4/23_path_UI_interactions_cookie_only_cookie_map_disqus_90'
path_UI_interactions_cookie_only_cookie_map_zync = 's3://shubh-sandbox/ai-audiences/q4/23_path_UI_interactions_cookie_only_cookie_map_zync_90'

path_UI_interactions_email_only_cookie_map_disqus = 's3://shubh-sandbox/ai-audiences/q4/24_path_UI_interactions_email_only_cookie_map_disqus_90'
path_UI_interactions_email_only_cookie_map_zync = 's3://shubh-sandbox/ai-audiences/q4/24_path_UI_interactions_email_only_cookie_map_zync_90'


# Functions

In [6]:
def generate_list_of_dates_to_process(date_start, date_end):
    
    start = datetime.datetime.strptime(date_start, '%Y-%m-%d')
    end = datetime.datetime.strptime(date_end, '%Y-%m-%d')
    list_of_dates_to_process = [start + datetime.timedelta(days=x) for x in range(0, (end - start).days)]

    return list_of_dates_to_process

In [7]:
# temp = generate_list_of_dates_to_process(date_start, date_end)
# # len(temp)
# # temp

In [8]:
def get_list_of_all_dates_present(bucket, prefix):

    s3 = boto3.client("s3")
    all_objects = s3.list_objects(Bucket=bucket,
                                  Prefix=prefix,
                                  Delimiter='/')

    list_date_buckets = []
    for o in all_objects.get('CommonPrefixes'):
        list_date_buckets.append(o.get('Prefix'))

    print('number of buckets in s3://{}/{} are {}'.format(bucket, prefix, len(list_date_buckets)))

    list_dates_present = [el.split('/')[-2].split('=')[-1] for el in list_date_buckets]

    list_dates_that_can_be_processed = []

    count = 0
    for el in list_dates_present:
        try:
            list_dates_that_can_be_processed.append(datetime.datetime.strptime(el, '%Y-%m-%d'))
        except:
            count = count + 1
            print('date partition that cannot be processed: {}'.format(el))

    print('number of buckets that cannot be processed in s3://{}/{} are {}'.format(bucket, prefix, count))
    print('number of buckets that can be processed in s3://{}/{} are {}'.format(bucket, prefix, len(
        list_dates_that_can_be_processed)))

    list_dates_that_can_be_processed.sort()

    return list_dates_that_can_be_processed


In [9]:
# parsed_url = urlparse(path_UI_COO_disqus, allow_fragments=False)
# bucket = parsed_url.netloc
# rest_of_path = parsed_url.path.lstrip('/')

# temp = get_list_of_all_dates_present(bucket, rest_of_path)
# # len(temp)
# # temp

In [10]:
def gen_path_list(path_UI_COO):

    bucket = path_UI_COO.split('/')[2]
    rest_of_path = os.path.join(*(path_UI_COO.split('/')[3:]))

    list_of_dates_present = get_list_of_all_dates_present(bucket=bucket,
                                                          prefix=rest_of_path)

    list_of_dates_to_process = generate_list_of_dates_to_process(date_start, date_end)
    
    list_of_dates_available = list(set(list_of_dates_to_process).intersection(list_of_dates_present))
    list_of_dates_available.sort()

    print(f'number of dates present are {len(list_of_dates_present)}')
    print(f'number of dates available are {len(list_of_dates_available)}')

    path_list = [path_UI_COO + 'dt=' + date.strftime('%Y-%m-%d') + '/*.parquet' for date in list_of_dates_available]

    print(len(path_list))
    
    return path_list

In [11]:
# temp = gen_path_list(path_UI_COO_disqus)
# # len(temp)
# # temp

In [12]:
def generate_ui_matrix(path_UI_COO, path_UI_interactions):
    path_list = gen_path_list(path_UI_COO)
    df = spark.read.option("inferSchema", True).parquet(*path_list)
    df = df.groupby('user_id', 'zcodes').agg(F.sum('count').alias('count'))
    df.write.parquet(path_UI_interactions)
    


# 11 - Create UI Tables X 3

In [ ]:
date_start = '2020-08-16'
date_end = '2020-11-16'

In [ ]:
path_UI_COO = path_UI_COO_disqus
path_UI_interactions = path_UI_interactions_disqus

generate_ui_matrix(path_UI_COO, path_UI_interactions)

In [ ]:
path_UI_COO = path_UI_COO_sizmek
path_UI_interactions = path_UI_interactions_sizmek

generate_ui_matrix(path_UI_COO, path_UI_interactions)

In [ ]:
path_UI_COO = path_UI_COO_zync
path_UI_interactions = path_UI_interactions_zync

generate_ui_matrix(path_UI_COO, path_UI_interactions)

# 10 UI -> Old UI Tables

In [ ]:
path_UI_interactions = path_UI_interactions_zync
path_UI_interactions_baseline = path_UI_interactions_baseline_zync

In [ ]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions+'/*.parquet')

In [ ]:
df.printSchema()

In [ ]:
df_cookie_cookie = spark.read.option("inferSchema", True).parquet(path_cookie_cookie+'/*.parquet')

In [ ]:
# df_cookie_cookie.printSchema()

In [ ]:
# print(f'number of entries: {df_cookie_cookie.count()}')

In [ ]:
# print(f'sizmek: {df_cookie_cookie.filter(df_cookie_cookie.source == "sizmek").count()}')
# print(f'zync: {df_cookie_cookie.filter(df_cookie_cookie.source == "zync").count()}')
# print(f'disqus: {df_cookie_cookie.filter(df_cookie_cookie.source == "disqus").count()}')

In [ ]:
# print(f'sizmek: {df_cookie_cookie.filter(df_cookie_cookie.destination == "sizmek").count()}')
# print(f'zync: {df_cookie_cookie.filter(df_cookie_cookie.destination == "zync").count()}')
# print(f'disqus: {df_cookie_cookie.filter(df_cookie_cookie.destination == "disqus").count()}')

In [ ]:
# print(f'zync->sizmek: {df_cookie_cookie.filter((df_cookie_cookie.source == "zync") & (df_cookie_cookie.destination == "sizmek")).count()}')
# print(f'disqus->sizmek: {df_cookie_cookie.filter((df_cookie_cookie.source == "disqus") & (df_cookie_cookie.destination == "sizmek")).count()}')
      

In [ ]:
# print(f'sizmek->zync: {df_cookie_cookie.filter((df_cookie_cookie.source == "sizmek") & (df_cookie_cookie.destination == "zync")).count()}')
# print(f'sizmek->disqus: {df_cookie_cookie.filter((df_cookie_cookie.source == "sizmek") & (df_cookie_cookie.destination == "disqus")).count()}')


In [ ]:
# print(f'zync->disqus: {df_cookie_cookie.filter((df_cookie_cookie.source == "zync") & (df_cookie_cookie.destination == "disqus")).count()}')
# print(f'disqus->zync: {df_cookie_cookie.filter((df_cookie_cookie.source == "disqus") & (df_cookie_cookie.destination == "zync")).count()}')


In [ ]:
print(path_UI_interactions)
print(path_UI_interactions_baseline)

In [ ]:
df_cookie_cookie = df_cookie_cookie.filter((df_cookie_cookie.source == "zync") & (df_cookie_cookie.destination == "sizmek"))
df = df.join(df_cookie_cookie, df.user_id==df_cookie_cookie.cookie_source, how='inner')
df = df.select(['cookie_destination', 'user_id', 'zcodes', 'count'])


In [ ]:
df.write.parquet(path_UI_interactions_baseline)

In [ ]:
path_UI_interactions_baseline

In [ ]:
# path_UI_interactions_baseline = path_UI_interactions_baseline_

In [ ]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions_baseline+'/*.parquet')

In [ ]:
df.printSchema()

In [ ]:
df.show(n=5)

In [ ]:
print(f'Number of entries/rows in the dataframe are {df.count()}')

cookie_destination = (df.select(F.countDistinct("cookie_destination")).collect())[0][0]
print(f'Number of distinct cookie_destination are {cookie_destination}')

count_distinct_cookies = (df.select(F.countDistinct("user_id")).collect())[0][0]
print(f'Number of distinct_cookies are {count_distinct_cookies}')

count_distinct_zcodes = (df.select(F.countDistinct("zcodes")).collect())[0][0]
print(f'Number of distinct_zcodes are {count_distinct_zcodes}')

count_distinct_cookie_cookie_pairs = df.dropDuplicates(['cookie_destination', 'user_id']).count()
print(f'Number of distinct_emails_cookies are {count_distinct_cookie_cookie_pairs}')

# 12 UI -> UI_email_cookie

In [ ]:
path_UI_interactions = path_UI_interactions_disqus
path_UI_interactions_email = path_UI_interactions_email_disqus


In [ ]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions+'/*.parquet')

In [ ]:
df.printSchema()

In [ ]:
# print(f'Number of entries/rows in the dataframe are {df.count()}')

# count_distinct_cookies = (df.select(F.countDistinct("user_id")).collect())[0][0]
# print(f'Number of distinct_cookies are {count_distinct_cookies}')

# count_distinct_zcodes = (df.select(F.countDistinct("zcodes")).collect())[0][0]
# print(f'Number of distinct_zcodes are {count_distinct_zcodes}')

In [ ]:
# df.show(n=5)

In [ ]:
df_cookie_email = spark.read.option("inferSchema", True).parquet(path_cookie_email+'/*.parquet')

In [ ]:
df_cookie_email.printSchema()

In [ ]:
# df_cookie_email.show(n=5)

In [ ]:
# print(f'number of entries: {df_cookie_email.count()}')
# print(f'sizmek: {df_cookie_email.filter(df_cookie_email.source == "sizmek").count()}')
# print(f'zync: {df_cookie_email.filter(df_cookie_email.source == "zync").count()}')
# print(f'disqus: {df_cookie_email.filter(df_cookie_email.source == "disqus").count()}')

In [ ]:
print(path_UI_interactions)
print(path_UI_interactions_email)

In [ ]:
df_cookie_email = df_cookie_email.filter(df_cookie_email.source == "disqus")
df = df.join(df_cookie_email, df.user_id==df_cookie_email.cookie_source, how='left')
df = df.select(['email_md5', 'user_id', 'zcodes', 'count'])

In [ ]:
df.write.parquet(path_UI_interactions_email)

In [ ]:
path_UI_interactions_email = path_UI_interactions_email_disqus

df = spark.read.option("inferSchema", True).parquet(path_UI_interactions_email+'/*.parquet')

In [ ]:
df.printSchema()

In [ ]:
df.show(n=2)

In [ ]:
print(f'Number of entries/rows in the dataframe are {df.count()}')

count_distinct_emails = (df.select(F.countDistinct("email_md5")).collect())[0][0]
print(f'Number of distinct_emails are {count_distinct_emails}')

count_distinct_cookies = (df.select(F.countDistinct("user_id")).collect())[0][0]
print(f'Number of distinct_cookies are {count_distinct_cookies}')

count_distinct_zcodes = (df.select(F.countDistinct("zcodes")).collect())[0][0]
print(f'Number of distinct_zcodes are {count_distinct_zcodes}')

count_distinct_email_cookie_pairs = df.dropDuplicates(['email_md5', 'user_id']).count()
print(f'Number of distinct_emails_cookies are {count_distinct_email_cookie_pairs}')

# 21 UI_email_cookie -> UI_cookie_only

In [ ]:
path_UI_interactions_email = path_UI_interactions_email_disqus
path_UI_interactions_cookie_only = path_UI_interactions_cookie_only_disqus

In [ ]:
print(path_UI_interactions_email)
print(path_UI_interactions_cookie_only)

In [ ]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions_email+'/*.parquet')

In [ ]:
df.printSchema()

In [ ]:
df = df.where(col("email_md5").isNull())


In [ ]:
df.write.parquet(path_UI_interactions_cookie_only)

In [ ]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions_cookie_only+'/*.parquet')

In [ ]:
df.printSchema()

In [ ]:
print(f'Number of entries/rows in the dataframe are {df.count()}')

count_distinct_emails = (df.select(F.countDistinct("email_md5")).collect())[0][0]
print(f'Number of distinct_emails are {count_distinct_emails}')

count_distinct_cookies = (df.select(F.countDistinct("user_id")).collect())[0][0]
print(f'Number of distinct_cookies are {count_distinct_cookies}')

count_distinct_zcodes = (df.select(F.countDistinct("zcodes")).collect())[0][0]
print(f'Number of distinct_zcodes are {count_distinct_zcodes}')

count_distinct_email_cookie_pairs = df.dropDuplicates(['email_md5', 'user_id']).count()
print(f'Number of distinct_emails_cookies are {count_distinct_email_cookie_pairs}')

# 22 UI_email_cookie -> UI_email_only

In [ ]:
path_UI_interactions_email = path_UI_interactions_email_disqus
path_UI_interactions_email_only = path_UI_interactions_email_only_disqus

In [ ]:
print(path_UI_interactions_email)
print(path_UI_interactions_email_only)

In [ ]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions_email+'/*.parquet')

In [ ]:
df.printSchema()

In [ ]:
df = df.where(col("email_md5").isNotNull())


In [ ]:
df.write.parquet(path_UI_interactions_email_only)

In [ ]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions_email_only+'/*.parquet')

In [ ]:
df.printSchema()

In [ ]:
print(f'Number of entries/rows in the dataframe are {df.count()}')

count_distinct_emails = (df.select(F.countDistinct("email_md5")).collect())[0][0]
print(f'Number of distinct_emails are {count_distinct_emails}')

count_distinct_cookies = (df.select(F.countDistinct("user_id")).collect())[0][0]
print(f'Number of distinct_cookies are {count_distinct_cookies}')

count_distinct_zcodes = (df.select(F.countDistinct("zcodes")).collect())[0][0]
print(f'Number of distinct_zcodes are {count_distinct_zcodes}')

count_distinct_email_cookie_pairs = df.dropDuplicates(['email_md5', 'user_id']).count()
print(f'Number of distinct_emails_cookies are {count_distinct_email_cookie_pairs}')

# 23 cookie_only -> cookie_only with cookie-cookie map

In [ ]:
path_UI_interactions_cookie_only = path_UI_interactions_cookie_only_disqus
path_UI_interactions_cookie_only_cookie_map = path_UI_interactions_cookie_only_cookie_map_disqus

In [ ]:
print(path_UI_interactions_cookie_only)
print(path_UI_interactions_cookie_only_cookie_map)

In [ ]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions_cookie_only+'/*.parquet')

In [ ]:
df.printSchema()

In [ ]:
df_cookie_cookie = spark.read.option("inferSchema", True).parquet(path_cookie_cookie+'/*.parquet')

In [ ]:
df_cookie_cookie.printSchema()

In [ ]:
print(path_UI_interactions_cookie_only)
print(path_UI_interactions_cookie_only_cookie_map)

In [ ]:
df_cookie_cookie = df_cookie_cookie.filter((df_cookie_cookie.source == "disqus") & (df_cookie_cookie.destination == "sizmek"))
df = df.join(df_cookie_cookie, df.user_id==df_cookie_cookie.cookie_source, how='inner')
df = df.select(['cookie_destination', 'email_md5', 'user_id', 'zcodes', 'count'])


In [ ]:
df.printSchema()

In [ ]:
df.write.parquet(path_UI_interactions_cookie_only_cookie_map)

In [ ]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions_cookie_only_cookie_map+'/*.parquet')


In [ ]:
print(f'Number of entries/rows in the dataframe are {df.count()}')

cookie_destination = (df.select(F.countDistinct("cookie_destination")).collect())[0][0]
print(f'Number of distinct cookie_destination are {cookie_destination}')

count_distinct_cookies = (df.select(F.countDistinct("user_id")).collect())[0][0]
print(f'Number of distinct_cookies are {count_distinct_cookies}')

count_distinct_zcodes = (df.select(F.countDistinct("zcodes")).collect())[0][0]
print(f'Number of distinct_zcodes are {count_distinct_zcodes}')

count_distinct_cookie_cookie_pairs = df.dropDuplicates(['cookie_destination', 'user_id']).count()
print(f'Number of distinct_emails_cookies are {count_distinct_cookie_cookie_pairs}')

# 24 email_only -> email_only with cookie-cookie map

In [25]:
path_UI_interactions_email_only = path_UI_interactions_email_only_disqus
path_UI_interactions_email_only_cookie_map = path_UI_interactions_email_only_cookie_map_disqus

In [26]:
print(path_UI_interactions_email_only)
print(path_UI_interactions_email_only_cookie_map)

s3://shubh-sandbox/ai-audiences/q4/22_path_UI_interactions_email_only_disqus_90
s3://shubh-sandbox/ai-audiences/q4/24_path_UI_interactions_email_only_cookie_map_disqus_90

In [27]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions_email_only+'/*.parquet')

In [28]:
df.printSchema()

root
 |-- email_md5: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- zcodes: string (nullable = true)
 |-- count: long (nullable = true)

In [29]:
df_cookie_cookie = spark.read.option("inferSchema", True).parquet(path_cookie_cookie+'/*.parquet')

In [30]:
df_cookie_cookie.printSchema()

root
 |-- source: string (nullable = true)
 |-- cookie_source: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- cookie_destination: string (nullable = true)
 |-- last_updated: timestamp (nullable = true)

In [31]:
print(path_UI_interactions_email_only)
print(path_UI_interactions_email_only_cookie_map)

s3://shubh-sandbox/ai-audiences/q4/22_path_UI_interactions_email_only_disqus_90
s3://shubh-sandbox/ai-audiences/q4/24_path_UI_interactions_email_only_cookie_map_disqus_90

In [32]:
df_cookie_cookie = df_cookie_cookie.filter((df_cookie_cookie.source == "disqus") & (df_cookie_cookie.destination == "sizmek"))
df = df.join(df_cookie_cookie, df.user_id==df_cookie_cookie.cookie_source, how='inner')
df = df.select(['cookie_destination', 'email_md5', 'user_id', 'zcodes', 'count'])


In [33]:
df.write.parquet(path_UI_interactions_email_only_cookie_map)

In [34]:
df = spark.read.option("inferSchema", True).parquet(path_UI_interactions_email_only_cookie_map+'/*.parquet')


In [35]:
print(f'Number of entries/rows in the dataframe are {df.count()}')

cookie_destination = (df.select(F.countDistinct("cookie_destination")).collect())[0][0]
print(f'Number of distinct cookie_destination are {cookie_destination}')

count_distinct_cookies = (df.select(F.countDistinct("user_id")).collect())[0][0]
print(f'Number of distinct_cookies are {count_distinct_cookies}')

count_distinct_zcodes = (df.select(F.countDistinct("zcodes")).collect())[0][0]
print(f'Number of distinct_zcodes are {count_distinct_zcodes}')

count_distinct_cookie_cookie_pairs = df.dropDuplicates(['cookie_destination', 'user_id']).count()
print(f'Number of distinct_emails_cookies are {count_distinct_cookie_cookie_pairs}')

Number of entries/rows in the dataframe are 6052127574
Number of distinct cookie_destination are 69191046
Number of distinct_cookies are 58385536
Number of distinct_zcodes are 754
Number of distinct_emails_cookies are 69384621